# Part II — Core Django  
## 8. Models and Database Basics (Schema, Migrations, Relationships, Integrity)

This chapter is where Django becomes a *real backend framework*: you stop hardcoding
data in Python lists and start storing and retrieving it from a database through
Django’s ORM.

You’ll learn:

- what Django **models** are (and what they are not)
- how Django maps models → database tables
- how **migrations** work (and why they are central to professional Django)
- how to design relationships (one-to-many, many-to-many, one-to-one)
- how to enforce data correctness with constraints
- how to create and inspect data using Django shell
- how to write tests around models

We’ll upgrade your existing `articles` app from in-memory data to real models.

---

## 8.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Explain how a Django model maps to a database table (fields → columns, rows → instances).
2. Create models with appropriate field types and options.
3. Create and apply migrations (`makemigrations`, `migrate`) and explain the difference.
4. Understand primary keys (`id`) and `BigAutoField`.
5. Create relationships:
   - `ForeignKey` (one-to-many)
   - `ManyToManyField` (many-to-many)
   - `OneToOneField` (one-to-one)
6. Use `Meta`, indexes, and constraints to enforce correctness at the database level.
7. Understand validation vs constraints (application-level vs database-level).
8. Use Django shell to create/query objects.
9. Write model tests using `TestCase`.

---

## 8.1 Database Fundamentals (Only What You Need for Django)

### 8.1.1 Tables, rows, columns (mapping to Django)
Relational databases (SQLite, PostgreSQL, MySQL) store data in tables.

- **Table**: collection of records for a concept (e.g., `articles_article`)
- **Row**: one record (e.g., one Article)
- **Column**: one attribute (e.g., title, slug)

Django mapping:

- Model class (e.g., `Article`) → Table
- Model instance (`Article(...)`) → Row
- Model field (`title = models.CharField(...)`) → Column

### 8.1.2 Primary keys (why every row needs an identity)
Most tables have a **primary key** column, typically an integer `id` that uniquely
identifies each row.

In Django:
- If you don’t define a primary key, Django creates one automatically (`id`).
- Modern Django defaults to `BigAutoField` (64-bit integer) for scalability.

---

## 8.2 SQLite vs PostgreSQL (Learning vs Production Reality)

### 8.2.1 SQLite (good for learning)
SQLite is file-based:
- simple setup (no server)
- great for local dev and tutorials

Limitations:
- concurrency constraints (write locking)
- fewer advanced features than PostgreSQL
- behavior differs in some edge cases

### 8.2.2 PostgreSQL (industry standard for Django production)
Most serious Django deployments use PostgreSQL because:
- robust concurrency
- advanced indexing and query planning
- strong integrity features
- JSON fields, full-text search, etc.

For this workbook:
- we use SQLite first to reduce setup friction
- later chapters show PostgreSQL best practices

---

## 8.3 What a Django Model Is (and what it contains)

A Django model is a Python class that:
- defines fields (schema)
- provides query API via the ORM
- can include behavior (methods)
- participates in validation and constraints

**Key mental model:**
- Models are not just schema; they are your *domain objects* + persistence mapping.

---

## 8.4 Designing Our Data Model (Articles + Tags)

Your current `articles/data.py` is a Python list. We’ll replace it with:

- `Tag` model
- `Article` model
- Many-to-many relationship: Article ↔ Tag

### 8.4.1 Why tags should be a model (not just a list of strings)
If tags are just strings embedded everywhere:
- you can’t easily enforce uniqueness (“Django” vs “django” duplicates)
- you can’t rename a tag safely
- you can’t query “all articles tagged X” efficiently without full scans

A `Tag` table gives:
- consistent canonical tags
- easy joins and queries
- data integrity options

---

## 8.5 Implement Models in `articles/models.py`

Edit `articles/models.py`:

```python
from django.db import models
from django.utils import timezone


class Tag(models.Model):
    name = models.CharField(max_length=50, unique=True)
    slug = models.SlugField(max_length=60, unique=True)

    class Meta:
        ordering = ["name"]

    def __str__(self) -> str:
        return self.name


class Article(models.Model):
    class Status(models.TextChoices):
        DRAFT = "draft", "Draft"
        PUBLISHED = "published", "Published"
        ARCHIVED = "archived", "Archived"

    title = models.CharField(max_length=200)
    slug = models.SlugField(max_length=220, unique=True)

    body = models.TextField()

    status = models.CharField(
        max_length=20,
        choices=Status.choices,
        default=Status.DRAFT,
    )

    # "published_at" is optional because drafts may not have it.
    published_at = models.DateTimeField(null=True, blank=True)

    # Many-to-many: one article can have many tags; one tag belongs to many articles.
    tags = models.ManyToManyField(Tag, related_name="articles", blank=True)

    created_at = models.DateTimeField(default=timezone.now, editable=False)
    updated_at = models.DateTimeField(auto_now=True)

    class Meta:
        ordering = ["-created_at"]
        indexes = [
            models.Index(fields=["slug"]),
            models.Index(fields=["status", "-created_at"]),
        ]
        constraints = [
            models.CheckConstraint(
                name="published_requires_published_at",
                check=(
                    models.Q(status="published", published_at__isnull=False)
                    | ~models.Q(status="published")
                ),
            ),
        ]

    def __str__(self) -> str:
        return self.title
```

### 8.5.1 Field-by-field explanation (what each does and why)

#### `CharField` vs `TextField`
- `CharField(max_length=...)` is for reasonably short strings (titles, names).
  - `max_length` is required and used by DB/schema and forms.
- `TextField()` is for long text (article body).
  - no `max_length` enforced at DB-level by default (you can validate in forms).

#### `SlugField`
A slug is a URL-friendly identifier: lowercase, hyphens, etc.

Why store `slug`?
- human-readable URLs: `/articles/hello-django/`
- stable identifiers for content
- avoids exposing numeric IDs in public URLs (not a security feature, but often nicer UX)

We set `unique=True` so two articles can’t share the same slug.

#### `choices` with `TextChoices` (professional pattern)
Using:

```python
class Status(models.TextChoices):
    DRAFT = "draft", "Draft"
```

gives:
- database stores stable values (`draft`, `published`)
- UI displays human labels (“Draft”)
- code is less error-prone than random strings

You’ll later use:
- `Article.Status.PUBLISHED`
instead of `"published"` scattered everywhere.

#### `null=True` vs `blank=True` (critical distinction)
- `null=True` affects the database column: it can store NULL.
- `blank=True` affects validation/forms: it can be empty in user input.

Common rule:
- For text fields, prefer `blank=True` and keep `null=False` (empty string instead of NULL).
- For dates/foreign keys, `null=True` is common if optional.

Here:
- `published_at` is optional → `null=True, blank=True`.

#### `ManyToManyField(Tag, related_name="articles")`
This creates a join table automatically (something like `articles_article_tags`).

- `related_name="articles"` means:
  - from a Tag instance, you can access `tag.articles.all()` to get all articles.
- `blank=True` means:
  - an article can be saved with no tags (common and convenient).

#### `auto_now=True` vs `default=timezone.now`
- `created_at = DateTimeField(default=timezone.now)`:
  - set once at creation time
  - remains stable
- `updated_at = DateTimeField(auto_now=True)`:
  - updates every time you save the model

This is a very common industry pattern.

#### Indexes
Indexes speed up queries at the cost of storage and slower writes.

We added:
- `Index(fields=["slug"])` because we often query by slug in detail pages.
- `Index(fields=["status", "-created_at"])` because list pages often filter by status and order by recency.

#### Constraints (data integrity at DB level)
We added a check:

- If status is `"published"`, then `published_at` must not be NULL.

Why this matters:
- Without it, your app could accidentally mark something published without timestamp.
- Constraints protect integrity even if data is created/modified outside your view logic.

---

## 8.6 Migrations (How Django Changes the Database Safely)

### 8.6.1 What a migration is
A migration is a versioned, replayable set of operations:
- create table
- add column
- alter column
- create index
- add constraint
- etc.

Migrations are tracked in your codebase and applied to databases in order.

**Industry reality:** migrations are as important as code. They are part of your deployment process.

### 8.6.2 `makemigrations` vs `migrate` (must understand)

- `python manage.py makemigrations`
  - compares your models to existing migration state
  - generates migration files (`articles/migrations/0001_initial.py`)
  - does NOT change the database

- `python manage.py migrate`
  - applies migrations to the database
  - changes schema (creates tables, columns, constraints)

### 8.6.3 Create migrations
Run:

```bash
python manage.py makemigrations
```

You should see output like:
- `Create model Tag`
- `Create model Article`

Now apply:

```bash
python manage.py migrate
```

### 8.6.4 Inspect generated SQL (very useful debugging skill)
Find the migration name Django created (e.g., `0001_initial`) and run:

```bash
python manage.py sqlmigrate articles 0001
```

This prints the SQL that Django will run (or ran).

Why this matters:
- helps you understand what schema is being created
- helps debug constraints/indexes
- builds confidence in what “models → DB” actually means

---

## 8.7 Use Django Shell to Create Data (Hands-on, Essential Skill)

Django shell loads your project and lets you interact with models.

Run:

```bash
python manage.py shell
```

Inside the shell:

```python
from articles.models import Article, Tag
from django.utils import timezone
```

### 8.7.1 Create tags
```python
django_tag = Tag.objects.create(name="Django", slug="django")
cbv_tag = Tag.objects.create(name="CBV", slug="cbv")
routing_tag = Tag.objects.create(name="Routing", slug="routing")
```

### 8.7.2 Create an article
```python
a1 = Article.objects.create(
    title="Hello Django",
    slug="hello-django",
    body="This is the first article stored in the database.",
    status=Article.Status.PUBLISHED,
    published_at=timezone.now(),
)
```

### 8.7.3 Add many-to-many tags (important: save first)
Many-to-many requires the Article to have a primary key first.

Since `a1` was created via `.create()`, it is already saved. Now:

```python
a1.tags.add(django_tag, routing_tag)
```

Verify tags:

```python
list(a1.tags.values_list("slug", flat=True))
# ['django', 'routing']
```

### 8.7.4 Query back articles
```python
Article.objects.all()
```

Get by slug:

```python
Article.objects.get(slug="hello-django")
```

Filter by status:

```python
Article.objects.filter(status=Article.Status.PUBLISHED)
```

Filter by tag relationship:

```python
Article.objects.filter(tags__slug="django").distinct()
```

#### What `tags__slug` means (preview of ORM)
`tags__slug` traverses the relationship:
- Article → tags (M2M) → Tag.slug

This is one of Django’s strongest features.

---

## 8.8 Replace In-Memory Data with Database Queries in Views

Now that the database has articles, update your `articles/views.py` to fetch from DB.

Edit `articles/views.py`:

```python
from django.http import Http404
from django.shortcuts import render
from django.views.decorators.http import require_GET

from .models import Article


@require_GET
def article_list_html(request):
    tag = request.GET.get("tag")
    q = request.GET.get("q")

    qs = Article.objects.all()

    # Optional: only show published items on the public page
    qs = qs.filter(status=Article.Status.PUBLISHED)

    if tag:
        qs = qs.filter(tags__slug=tag)

    if q:
        qs = qs.filter(title__icontains=q) | qs.filter(body__icontains=q)

    qs = qs.distinct()

    return render(
        request,
        "articles/list.html",
        {
            "articles": qs,
            "tag": tag,
            "q": q,
        },
    )


@require_GET
def article_detail_html(request, slug: str):
    try:
        article = Article.objects.get(slug=slug, status=Article.Status.PUBLISHED)
    except Article.DoesNotExist as e:
        raise Http404("Article not found") from e

    return render(
        request,
        "articles/detail.html",
        {
            "article": article,
        },
    )
```

### 8.8.1 Important explanation: QuerySets vs lists
`Article.objects.all()` returns a **QuerySet**, not a list.

A QuerySet is:
- lazy (it doesn’t hit the DB immediately)
- composable (you can keep adding filters)
- evaluated when needed (iterated, converted to list, rendered)

This laziness is key to Django performance and design; you’ll study it deeply in the ORM chapter.

### 8.8.2 Fix the `q` filter (better version)
The line:

```python
qs = qs.filter(title__icontains=q) | qs.filter(body__icontains=q)
```

works, but is less idiomatic. A better pattern uses `Q` objects:

```python
from django.db.models import Q
...
if q:
    qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))
```

We’ll formalize this in the ORM chapter; consider this a preview.

---

## 8.9 Update Templates to Use Model Instances (Not Dicts)

Your `articles/list.html` currently assumes dict keys (`a.title`, `a.slug` still works
in templates for both dicts and objects, but tags differ).

Model relationships are accessed via managers:

- `article.tags.all` is a queryset method call (so you iterate `article.tags.all`)

Update `articles/templates/articles/list.html`:

```django
{% extends "base.html" %}

{% block title %}Articles{% endblock %}

{% block content %}
  <h1>Articles</h1>

  <ul>
    {% for a in articles %}
      <li>
        <strong>
          <a href="{% url 'articles:detail_html' slug=a.slug %}">
            {{ a.title }}
          </a>
        </strong>

        <div>
          status: <code>{{ a.status }}</code>
        </div>

        <div>
          tags:
          {% for t in a.tags.all %}
            <code>{{ t.slug }}</code>
          {% empty %}
            <em>(no tags)</em>
          {% endfor %}
        </div>

        <p>{{ a.body|truncatechars:80 }}</p>
      </li>
    {% empty %}
      <li>No articles found.</li>
    {% endfor %}
  </ul>
{% endblock %}
```

### Add a detail template
Create `articles/templates/articles/detail.html`:

```django
{% extends "base.html" %}

{% block title %}{{ article.title }}{% endblock %}

{% block content %}
  <article>
    <h1>{{ article.title }}</h1>

    <p>
      slug: <code>{{ article.slug }}</code>
      • status: <code>{{ article.status }}</code>
    </p>

    <p>
      published:
      {% if article.published_at %}
        {{ article.published_at }}
      {% else %}
        (not published)
      {% endif %}
    </p>

    <div>
      tags:
      {% for t in article.tags.all %}
        <code>{{ t.name }}</code>
      {% empty %}
        <em>(no tags)</em>
      {% endfor %}
    </div>

    <hr />

    <div>
      {{ article.body }}
    </div>
  </article>
{% endblock %}
```

### Wire the new URL
Update `articles/urls.py`:

```python
from django.urls import path

from . import views

app_name = "articles"

urlpatterns = [
    path("html/", views.article_list_html, name="list_html"),
    path("html/<slug:slug>/", views.article_detail_html, name="detail_html"),
]
```

---

## 8.10 Validation vs Constraints (Two Layers of “Correctness”)

### 8.10.1 Validation (application layer)
Django can validate model fields when you call:

```python
article.full_clean()
```

But Django does **not** automatically run `full_clean()` on every `save()`.

Where validation typically runs:
- Django Forms / ModelForms (when you build web forms)
- DRF serializers (for APIs)
- explicit `full_clean()` in service logic (optional, case-dependent)

### 8.10.2 Constraints (database layer)
Constraints (like `unique=True`, CheckConstraint) are enforced by the DB.

Why constraints matter:
- they protect you even if data is created by:
  - scripts
  - admin mistakes
  - management commands
  - concurrent requests
  - direct DB operations

**Industry standard:** use database constraints for critical invariants.

---

## 8.11 Relationship Types (ForeignKey, OneToOne, ManyToMany) — With Practical Examples

### 8.11.1 ForeignKey (one-to-many)
Example: Many articles belong to one author.

If you use Django’s built-in User model:

```python
from django.conf import settings

author = models.ForeignKey(
    settings.AUTH_USER_MODEL,
    on_delete=models.PROTECT,
    related_name="articles",
)
```

Explanation:
- Many Article rows can point to one User row.
- `on_delete=models.PROTECT` prevents deleting a user who still has articles (good
  for preserving authored content).
- `related_name="articles"` allows `user.articles.all()`.

### 8.11.2 OneToOne (one-to-one)
Example: user profile.

```python
profile = models.OneToOneField(
    settings.AUTH_USER_MODEL,
    on_delete=models.CASCADE,
    related_name="profile",
)
```

Meaning:
- each user has at most one profile row
- profile belongs to exactly one user

### 8.11.3 ManyToMany (many-to-many)
We already used it for tags.

Under the hood:
- a join table stores pairs (article_id, tag_id)

Why this is better than “comma-separated tags”:
- queryable
- indexable
- consistent and normalized

---

## 8.12 Common Migration Mistakes (and Fixes)

### Mistake A: “I changed models but forgot to makemigrations”
Symptom:
- code expects new field but DB doesn’t have it
- runtime errors like “no such column”

Fix:
```bash
python manage.py makemigrations
python manage.py migrate
```

### Mistake B: Deleting migration files casually
In real projects, migration history is part of team coordination.
Deleting migrations can break other environments.

Safe approach for learning:
- If you’re early and don’t care about data, you can reset DB + migrations.
Professional approach:
- create new migrations to modify schema; don’t rewrite history unless you know
  exactly what you’re doing.

### Mistake C: Circular dependencies across apps
Happens when two apps reference each other in models.
Fix patterns:
- use `settings.AUTH_USER_MODEL` for user references
- use string references `"app.ModelName"`
- refactor relationships into one direction or separate app

We’ll address this when you build multi-app systems.

---

## 8.13 Testing Models (Create Data, Assert Behavior)

Create `articles/tests_models.py` (or add to `articles/tests.py`):

```python
from django.test import TestCase
from django.utils import timezone

from .models import Article, Tag


class ArticleModelTests(TestCase):
    def test_can_create_article_with_tags(self):
        django_tag = Tag.objects.create(name="Django", slug="django")

        article = Article.objects.create(
            title="Test",
            slug="test",
            body="Body",
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
        )
        article.tags.add(django_tag)

        self.assertEqual(article.tags.count(), 1)
        self.assertEqual(article.tags.first().slug, "django")

    def test_published_requires_published_at_constraint(self):
        # This test demonstrates the DB constraint.
        # Depending on DB backend, the exception type may vary, but IntegrityError
        # is typical.
        from django.db import IntegrityError

        with self.assertRaises(IntegrityError):
            Article.objects.create(
                title="Invalid",
                slug="invalid",
                body="Body",
                status=Article.Status.PUBLISHED,
                published_at=None,
            )
```

Run:

```bash
python manage.py test
```

---

## 8.14 Mini-Lab (End-to-End): Seed Data via Shell and Render Pages

1. Run migrations (if not already):
```bash
python manage.py makemigrations
python manage.py migrate
```

2. Seed a couple tags and articles in `python manage.py shell` (as shown earlier).

3. Run server:
```bash
python manage.py runserver
```

4. Visit:
- `/articles/html/`
- `/articles/html/?tag=django`
- `/articles/html/<slug>/`

You should see real DB-backed content.

---

## 8.15 Exercises (Do These Before Proceeding)

1. Add an `excerpt` field to Article:
   - `excerpt = models.CharField(max_length=300, blank=True)`
   - Create migrations, migrate
   - Update templates to display excerpt if present, else fallback to truncating body

2. Add an `is_featured` boolean:
   - default False
   - Add an index on it (or include in a composite index)
   - Update list page to optionally filter featured via `?featured=true`

3. Add a uniqueness rule:
   - make `(title, status)` unique *or* add a unique constraint on `(slug, status)`
   - explain why you chose it (what does uniqueness mean for your domain?)

4. Use `sqlmigrate` to inspect the migration SQL and write 3 bullet points about
   what tables/indexes/constraints were created.

---

## 8.16 Chapter Summary

- Models define schema + behavior; fields map to columns; instances map to rows.
- Migrations are the versioned history of your schema.
  - `makemigrations` generates migration files
  - `migrate` applies them to DB
- Relationships are core to real apps:
  - FK, M2M, O2O
- Constraints and indexes are database-level tools for correctness and performance.
- Django shell is your fastest learning/debugging tool for models.
- Tests should prove your model rules hold (especially constraints).

---

Next chapter: **Part II — Chapter 9: Django ORM (Querying Like a Pro)**  
We’ll go deep into QuerySets, filtering, ordering, annotations, joins, `Q` objects,
`select_related`/`prefetch_related`, and performance patterns—using the models you
just built.